<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

In this example, we load the data from a csv file with `pandas.read_csv`, and we use the [iter_pandas](https://riverml.xyz/latest/api/stream/iter-pandas/) utility method to iterate over the `DataFrame`.

In [3]:
stream = iter_pandas(X=data[features], y=data['rain'])

## Naïve Bayes
---
[GaussianNB](https://riverml.xyz/latest/api/naive-bayes/GaussianNB/) maintains a Gaussian distribution $G_{cf}$ is maintained for each class $c$ and each feature $f$. Each Gaussian is updated using the amount associated with each feature; the details can be be found in proba.Gaussian. The joint log-likelihood is then obtained by summing the log probabilities of each feature associated with each class.

In [4]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 71.27%, BalancedAccuracy: 71.96%, GeometricMean: 0.719288, CohenKappa: 0.401927
[2,000] Accuracy: 69.88%, BalancedAccuracy: 70.38%, GeometricMean: 0.703702, CohenKappa: 0.36229
[3,000] Accuracy: 68.99%, BalancedAccuracy: 69.64%, GeometricMean: 0.696182, CohenKappa: 0.34233
[4,000] Accuracy: 68.82%, BalancedAccuracy: 68.87%, GeometricMean: 0.688653, CohenKappa: 0.334827
[5,000] Accuracy: 69.09%, BalancedAccuracy: 67.97%, GeometricMean: 0.679167, CohenKappa: 0.327016
[6,000] Accuracy: 69.13%, BalancedAccuracy: 67.87%, GeometricMean: 0.677979, CohenKappa: 0.326471
[7,000] Accuracy: 69.15%, BalancedAccuracy: 67.89%, GeometricMean: 0.678197, CohenKappa: 0.326151
[8,000] Accuracy: 68.50%, BalancedAccuracy: 67.31%, GeometricMean: 0.672467, CohenKappa: 0.315621
[9,000] Accuracy: 68.65%, BalancedAccuracy: 66.69%, GeometricMean: 0.665023, CohenKappa: 0.309668
[10,000] Accuracy: 69.04%, BalancedAccuracy: 66.36%, GeometricMean: 0.660147, CohenKappa: 0.307472
[11,000] Accuracy: 69

Accuracy: 69.21%, BalancedAccuracy: 66.27%, GeometricMean: 0.657997, CohenKappa: 0.312843

## K-Nearest Neighbors
---
[KNN](https://riverml.xyz/latest/api/neighbors/KNNClassifier/) is a non-parametric classification method that keeps track of the last window_size training samples. The predicted class-label for a given query sample is obtained in two steps:

- Find the closest n_neighbors to the query sample in the data window. 
- Aggregate the class-labels of the n_neighbors to define the predicted class for the query sample.

In [5]:
from river.neighbors import KNNClassifier

model = KNNClassifier(n_neighbors=5, window_size=1000)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 77.18%, BalancedAccuracy: 71.52%, GeometricMean: 0.696328, CohenKappa: 0.453244
[2,000] Accuracy: 78.34%, BalancedAccuracy: 71.07%, GeometricMean: 0.687121, CohenKappa: 0.449479
[3,000] Accuracy: 78.86%, BalancedAccuracy: 70.49%, GeometricMean: 0.676328, CohenKappa: 0.443403
[4,000] Accuracy: 78.29%, BalancedAccuracy: 70.44%, GeometricMean: 0.677807, CohenKappa: 0.439294
[5,000] Accuracy: 78.06%, BalancedAccuracy: 70.37%, GeometricMean: 0.67768, CohenKappa: 0.436813
[6,000] Accuracy: 77.95%, BalancedAccuracy: 70.53%, GeometricMean: 0.680824, CohenKappa: 0.438008
[7,000] Accuracy: 78.24%, BalancedAccuracy: 70.95%, GeometricMean: 0.686521, CohenKappa: 0.44553
[8,000] Accuracy: 77.96%, BalancedAccuracy: 70.85%, GeometricMean: 0.686013, CohenKappa: 0.442612
[9,000] Accuracy: 78.12%, BalancedAccuracy: 71.08%, GeometricMean: 0.688297, CohenKappa: 0.448102
[10,000] Accuracy: 78.16%, BalancedAccuracy: 71.12%, GeometricMean: 0.689098, CohenKappa: 0.448404
[11,000] Accuracy: 78

Accuracy: 77.91%, BalancedAccuracy: 72.06%, GeometricMean: 0.703281, CohenKappa: 0.462426

## Hoeffding Tree
---

[Hoeffding Tree](https://riverml.xyz/latest/api/tree/HoeffdingTreeClassifier/) 

Tree-based models are popular due to their interpretability. They use a tree data structure to model the data. When a sample arrives, it traverses the tree until it reaches a leaf node. Internal nodes define the path for a data sample based on the values of its features. Leaf nodes are models that provide predictions for unlabeled-samples and can update their internal state using the labels from labeled samples.

In [6]:
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier()
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.87%, BalancedAccuracy: 71.11%, GeometricMean: 0.711018, CohenKappa: 0.389171
[2,000] Accuracy: 69.73%, BalancedAccuracy: 68.12%, GeometricMean: 0.680063, CohenKappa: 0.334537
[3,000] Accuracy: 70.89%, BalancedAccuracy: 63.00%, GeometricMean: 0.601571, CohenKappa: 0.268525
[4,000] Accuracy: 71.29%, BalancedAccuracy: 61.85%, GeometricMean: 0.574005, CohenKappa: 0.255662
[5,000] Accuracy: 71.79%, BalancedAccuracy: 62.23%, GeometricMean: 0.575773, CohenKappa: 0.265883
[6,000] Accuracy: 72.13%, BalancedAccuracy: 62.56%, GeometricMean: 0.578828, CohenKappa: 0.273963
[7,000] Accuracy: 72.82%, BalancedAccuracy: 64.11%, GeometricMean: 0.6042, CohenKappa: 0.30228
[8,000] Accuracy: 72.58%, BalancedAccuracy: 64.31%, GeometricMean: 0.608998, CohenKappa: 0.304549
[9,000] Accuracy: 72.80%, BalancedAccuracy: 63.98%, GeometricMean: 0.599835, CohenKappa: 0.302127
[10,000] Accuracy: 72.85%, BalancedAccuracy: 63.64%, GeometricMean: 0.593219, CohenKappa: 0.29692
[11,000] Accuracy: 73.3

Accuracy: 73.55%, BalancedAccuracy: 65.87%, GeometricMean: 0.625562, CohenKappa: 0.340681

## Hoeffding Adaptive Tree
---
The [HAT](https://riverml.xyz/latest/api/tree/HoeffdingAdaptiveTreeClassifier/) model uses `ADWIN` to detect changes. If change is detected in a given branch, an alternate branch is created and eventually replaces the original branch if it shows better performance on new data.

In [7]:
from hoeffding_adaptive_tree_classifier import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 68.37%, BalancedAccuracy: 65.29%, GeometricMean: 0.646908, CohenKappa: 0.298324
[2,000] Accuracy: 69.48%, BalancedAccuracy: 63.36%, GeometricMean: 0.614911, CohenKappa: 0.26877
[3,000] Accuracy: 71.09%, BalancedAccuracy: 60.64%, GeometricMean: 0.553518, CohenKappa: 0.232311
[4,000] Accuracy: 72.02%, BalancedAccuracy: 62.12%, GeometricMean: 0.572311, CohenKappa: 0.264893
[5,000] Accuracy: 72.85%, BalancedAccuracy: 63.23%, GeometricMean: 0.586106, CohenKappa: 0.289428
[6,000] Accuracy: 73.33%, BalancedAccuracy: 64.41%, GeometricMean: 0.604926, CohenKappa: 0.311497
[7,000] Accuracy: 73.91%, BalancedAccuracy: 65.10%, GeometricMean: 0.613959, CohenKappa: 0.325733
[8,000] Accuracy: 73.51%, BalancedAccuracy: 64.66%, GeometricMean: 0.607665, CohenKappa: 0.317075
[9,000] Accuracy: 73.81%, BalancedAccuracy: 64.68%, GeometricMean: 0.604427, CohenKappa: 0.320774
[10,000] Accuracy: 73.85%, BalancedAccuracy: 64.78%, GeometricMean: 0.606611, CohenKappa: 0.321975
[11,000] Accuracy: 7

Accuracy: 73.73%, BalancedAccuracy: 66.59%, GeometricMean: 0.63769, CohenKappa: 0.352226

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [8]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## Naïve Bayes

In [9]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 83.98%, BalancedAccuracy: 77.03%, GeometricMean: 0.741456, CohenKappa: 0.6
[2,000] Accuracy: 86.29%, BalancedAccuracy: 80.38%, GeometricMean: 0.782075, CohenKappa: 0.665697
[3,000] Accuracy: 87.00%, BalancedAccuracy: 81.06%, GeometricMean: 0.790111, CohenKappa: 0.680837
[4,000] Accuracy: 87.55%, BalancedAccuracy: 81.46%, GeometricMean: 0.794563, CohenKappa: 0.6909
[5,000] Accuracy: 87.42%, BalancedAccuracy: 81.35%, GeometricMean: 0.79319, CohenKappa: 0.688606
[6,000] Accuracy: 80.50%, BalancedAccuracy: 75.06%, GeometricMean: 0.717605, CohenKappa: 0.546748
[7,000] Accuracy: 74.71%, BalancedAccuracy: 70.42%, GeometricMean: 0.661399, CohenKappa: 0.438805
[8,000] Accuracy: 70.87%, BalancedAccuracy: 67.68%, GeometricMean: 0.62937, CohenKappa: 0.373843
[9,000] Accuracy: 68.01%, BalancedAccuracy: 65.87%, GeometricMean: 0.609456, CohenKappa: 0.329905
[10,000] Accuracy: 66.25%, BalancedAccuracy: 64.90%, GeometricMean: 0.602605, CohenKappa: 0.305641
[11,000] Accuracy: 66.75%, B

Accuracy: 65.94%, BalancedAccuracy: 64.90%, GeometricMean: 0.617772, CohenKappa: 0.303782

## KNN with ADWIN
---

This classifier is an improvement from the regular kNN method, as it is resistant to concept drift. It uses the ADWIN change detector to decide which samples to keep and which ones to forget, and by doing so it regulates the sample window size.

In [10]:
from river.neighbors import KNNADWINClassifier
from river import compose

model = (
    compose.Discard('elevel', 'car', 'zipcode') |
    KNNADWINClassifier(n_neighbors=5, window_size=1000)
)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 58.16%, BalancedAccuracy: 49.55%, GeometricMean: 0.422869, CohenKappa: -0.009662
[2,000] Accuracy: 58.08%, BalancedAccuracy: 50.04%, GeometricMean: 0.432183, CohenKappa: 0.000878
[3,000] Accuracy: 58.72%, BalancedAccuracy: 50.23%, GeometricMean: 0.430091, CohenKappa: 0.004976
[4,000] Accuracy: 59.56%, BalancedAccuracy: 50.57%, GeometricMean: 0.430737, CohenKappa: 0.012374
[5,000] Accuracy: 59.99%, BalancedAccuracy: 51.15%, GeometricMean: 0.438449, CohenKappa: 0.024936
[6,000] Accuracy: 59.46%, BalancedAccuracy: 54.00%, GeometricMean: 0.492866, CohenKappa: 0.08485
[7,000] Accuracy: 60.55%, BalancedAccuracy: 57.60%, GeometricMean: 0.551442, CohenKappa: 0.157202
[8,000] Accuracy: 61.30%, BalancedAccuracy: 59.63%, GeometricMean: 0.581953, CohenKappa: 0.196598
[9,000] Accuracy: 61.98%, BalancedAccuracy: 61.11%, GeometricMean: 0.602393, CohenKappa: 0.224807
[10,000] Accuracy: 62.32%, BalancedAccuracy: 61.87%, GeometricMean: 0.613391, CohenKappa: 0.238926
[11,000] Accuracy: 

Accuracy: 64.41%, BalancedAccuracy: 64.04%, GeometricMean: 0.636468, CohenKappa: 0.282327

## Hoeffding Tree

In [12]:
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 81.68%, BalancedAccuracy: 74.40%, GeometricMean: 0.711195, CohenKappa: 0.542118
[2,000] Accuracy: 82.29%, BalancedAccuracy: 78.91%, GeometricMean: 0.781873, CohenKappa: 0.594572
[3,000] Accuracy: 83.49%, BalancedAccuracy: 82.29%, GeometricMean: 0.822033, CohenKappa: 0.635939
[4,000] Accuracy: 85.17%, BalancedAccuracy: 84.61%, GeometricMean: 0.845969, CohenKappa: 0.673779
[5,000] Accuracy: 86.20%, BalancedAccuracy: 86.08%, GeometricMean: 0.860795, CohenKappa: 0.698778
[6,000] Accuracy: 80.13%, BalancedAccuracy: 79.01%, GeometricMean: 0.788828, CohenKappa: 0.578215
[7,000] Accuracy: 76.30%, BalancedAccuracy: 75.31%, GeometricMean: 0.751008, CohenKappa: 0.508355
[8,000] Accuracy: 74.11%, BalancedAccuracy: 73.44%, GeometricMean: 0.732605, CohenKappa: 0.471077
[9,000] Accuracy: 73.64%, BalancedAccuracy: 73.16%, GeometricMean: 0.729417, CohenKappa: 0.465858
[10,000] Accuracy: 73.95%, BalancedAccuracy: 73.64%, GeometricMean: 0.734276, CohenKappa: 0.474926
[11,000] Accuracy: 

Accuracy: 71.70%, BalancedAccuracy: 71.71%, GeometricMean: 0.717109, CohenKappa: 0.433489

## Hoeffding Adaptive Tree

In [11]:
from hoeffding_adaptive_tree_classifier import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'], seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 83.28%, BalancedAccuracy: 77.78%, GeometricMean: 0.760039, CohenKappa: 0.596293
[2,000] Accuracy: 87.29%, BalancedAccuracy: 85.33%, GeometricMean: 0.851014, CohenKappa: 0.714108
[3,000] Accuracy: 88.66%, BalancedAccuracy: 87.84%, GeometricMean: 0.87806, CohenKappa: 0.748789
[4,000] Accuracy: 90.02%, BalancedAccuracy: 89.65%, GeometricMean: 0.896477, CohenKappa: 0.778601
[5,000] Accuracy: 90.52%, BalancedAccuracy: 90.48%, GeometricMean: 0.904801, CohenKappa: 0.791178
[6,000] Accuracy: 85.18%, BalancedAccuracy: 84.58%, GeometricMean: 0.845457, CohenKappa: 0.686609
[7,000] Accuracy: 81.98%, BalancedAccuracy: 81.70%, GeometricMean: 0.816877, CohenKappa: 0.630195
[8,000] Accuracy: 80.12%, BalancedAccuracy: 80.11%, GeometricMean: 0.801098, CohenKappa: 0.598378
[9,000] Accuracy: 78.84%, BalancedAccuracy: 78.80%, GeometricMean: 0.787972, CohenKappa: 0.574675
[10,000] Accuracy: 77.66%, BalancedAccuracy: 77.53%, GeometricMean: 0.775013, CohenKappa: 0.551273
[11,000] Accuracy: 7

Accuracy: 76.01%, BalancedAccuracy: 75.65%, GeometricMean: 0.753245, CohenKappa: 0.515992

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.